# Data preparation

## Creating purchases columns in user_id - SQL method

**Creation de la colonne purchases**

On créé une colonne "purchases" pour spécifier si l'utilisateur à déjà acheté au moins un produit.  
On le fait via la console GCP avec les commande SQL ci dessous.  

Dans un premier temps on doit créé la colonne "purchases" de type booléen qui sera vide.  
Puis on initialise toutes les valeurs à False avec la requête ci dessous.

```   
UPDATE `marbotic.marbotic_dataset.user_id_f`  
SET purchases = False  
WHERE TRUE  
```


On mets à jour les profils utilisateurs ayant effectué au moins un achat avec la requête ci dessous.  

```
UPDATE `marbotic.marbotic_dataset.user_id_f`   
SET purchases = True  
WHERE Purchases_PY1Y = 1  
OR  
Purchases_PLY = 1  
OR   
Purchases_MEGR = 1  
OR  
Purchases_PLM = 1  
OR   
Purchases_ME1 = 1  
OR   
Purchases_EY3M = 1  
OR  
Purchases_tier_upgrade_0_to_1 = 1  
OR  
Purchases_PM3M = 1  
OR  
Purchases_MPGR = 1  
OR  
Purchases_EY1Y = 1  
OR  
Purchases_PY3M = 1  
```

Enfin on ajoute cette colonne à la table users_metrics 

**Vérifier s'il faut créer la colonne à la main avant**

```
UPDATE `marbotic.marbotic_dataset.users_metrics_f` as um
SET um.purchases = tmp.purchases
FROM (SELECT um.user_id, MAX(ui.purchases) as purchases
FROM `marbotic.marbotic_dataset.users_metrics_f` as um LEFT JOIN `marbotic.marbotic_dataset.user_id_f` as ui
ON um.user_id = ui.User_id
GROUP BY um.user_id
) as tmp
WHERE um.user_id = tmp.user_id
```


## Creating game_events and setup_events in session_metrics - SQL method

**Creation de la colonne game_events**

On créé une colonne "game_events" pour compter le nombre d'event de jeu par session.  
On le fait via la console GCP avec les commande SQL ci dessous.  

Dans un premier temps on doit créé la colonne "game_events" de type Float qui sera vide.  
Puis on la complète en sommant les events de jeu

```
UPDATE `marbotic.marbotic_dataset.sessions_metrics_f` 
SET game_events = 
 event_type_Activation_Game + event_type_Game_Stop + event_type_Scaffolding_Reset + event_type_Game_Pause + event_type_Game_Start + 
 event_type_Error_Error + event_type_Scene_Enter + event_type_Scene_Leave + event_type_Action_Click + event_type_Activity_Stop + 
 event_type_Activity_Start + event_type_Activity_Good_Answer + event_type_Activity_Wrong_Answer + event_type_Scaffolding_Scaffolding
WHERE True
```

**Creation de la colonne setup_event**

De la même façon on crée la colonne setup_event pour compter le nombre d'event de setup par session.  
On le fait via la console GCP avec les commande SQL ci dessous.  

Dans un premier temps on doit créé la colonne "setup_events" de type Float qui sera vide.  
Puis on la complète en sommant les events de setup

```
UPDATE `marbotic.marbotic_dataset.sessions_metrics` 
SET setup_events = 
event_type_Profile_Create + 
event_type_Profile_Update +
event_type_Account_Sign_In +
event_type_OB_ProfileName +
event_type_OB_ProfileBirthdate +
event_type_OB_KeyRedeem +
event_type_OB_KeyFail +
event_type_OB_RenewalSelection +
event_type_Account_Sign_Up +
event_type_OB_ProductSelection +
event_type_OB_ProductSK +
event_type_OB_ProductHelp +
event_type_OB_ProductActivation +
event_type_OB_RenewalSuccess+
event_type_OB_ProductSeeMore+
event_type_OB_ProductOther+
event_type_Account_Sign_Out+
event_type_OB_CommPreview+
event_type_OB_DoYouHaveKit+
event_type_OB_HasKitYes+
event_type_OB_Email+
event_type_OB_VerifCode+
event_type_Activation_New_Activation+
event_type_Activation_Product+
event_type_Activation_Piece+
event_type_OB_Success+
event_type_OB_RenewalFail+
event_type_OB_ClickTrial+
event_type_Popup_Open+
event_type_Popup_Close+
event_type_OB_HasKitNo+
event_type_OB_DiscoverTrial+
event_type_Account_Update_Request+
event_type_Account_Update_Success+
event_type_Profile_Delete+
event_type_OB_TrialStart+
event_type_OB_ClickStore+
event_type_OB_HasKeyNo+
event_type_OB_Type+
event_type_OB_KeySuccess+
event_type_Profile_Reset_Settings_and_Progression+
event_type_Action_ConversionClickRedeem+
event_type_Action_ConversionClickSubscribe+
event_type_Toast_Appear+
event_type_Toast_Disappear
WHERE True
```

## Creating table session_augmented